<a href="https://colab.research.google.com/github/YonggunJung/ICT_TensorFlow/blob/main/20231010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 먼저 현재 Google Colab에서 제공하는 CUDA버젼과 cuDDN버전 확인

!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
# 현재 Graphic Driver의 버전과 해당버전에 맞는 CUDA추천 버전 확인


!nvidia-smi # NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0

Tue Oct 10 12:59:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch

print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

# cuda version: 11.8
# cudnn version:8.7.00 (cuDNN은 cuda버전과 dependency가 있어요!)

cuda version: 11.8
cudnn version:8700


In [5]:
# 우리는 Tensorflow Object Detection API를 이용해서
# 한땀한땀 model을 구현X

# 결국 Tensorflow가 제공하는 Object Detection Model을 이용
# Tensorflow Model Garden에서 제공하는 모델을 가져다가 사용
# Tensorflow Model Garden은 최신 tensorflow model들이 옹기종기 모여있는 저장소예요(Repository)

In [6]:
# 제공받은 Disk에 폴더를 하나 만들어요!
!mkdir /content/TF2_Object_Detection_API

mkdir: cannot create directory ‘/content/TF2_Object_Detection_API’: File exists


In [7]:
# 해당폴더로 working directory로 이동
# cd 명령어는 주석과 함께 쓰지 않는다

In [8]:
cd /content/TF2_Object_Detection_API

/content/TF2_Object_Detection_API


In [9]:
# Tensorflow Model Garden에서 Object Detection과 연관된 model을 다운로드
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [10]:
# working directory를 models/research로 변경

In [11]:
cd /content/TF2_Object_Detection_API/models/research

/content/TF2_Object_Detection_API/models/research


In [12]:
# 복사한 model에서 필요한 부분을 설치
# Tensorflow Object Detection API 설치(package등등)
# cp는 복사 복사할 파일 + 복사할 위치

!cp /content/TF2_Object_Detection_API/models/research/object_detection/packages/tf2/setup.py /content/TF2_Object_Detection_API/models/research

In [13]:
# 필요한 package설치

!python -m pip install .

Processing /content/TF2_Object_Detection_API/models/research
  Preparing metadata (setup.py) ... done
  Using cached tensorflow-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (489.8 MB)
  Using cached tensorboard-2.14.1-py3-none-any.whl (5.5 MB)
  Using cached tensorflow_estimator-2.14.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.14.0-py3-none-any.whl (1.7 MB)
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1697354 sha256=792ea44b663244be046ae072dc03e0ca17ec5b4e4869148b01b8279324b70d01
  Stored in directory: /tmp/pip-ephem-wheel-cache-ycow7zd0/wheels/69/56/7a/910555172ac10b761e7408c23079449fea424254ff5296fee6
Successfully built object-detection
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: ke

In [14]:
# 나중에 학습할 데이터 셋 준비
# 데이터 셋은 TFRecord로 만들어야함
# TFRecord는 Tensorflow에서 지원하는 데이터를 표현하기 위한 방법
# TFRecord는 Tensorflow의 표준 데이터 파일 포맷
# TFRecord사용 이유
# 1. 파일 하나에 이미지 데이터와 label데이터를 같이 포함. 바이너리 포맷이어서 오버헤드를 줄일 수 있음
# 2. 파일의 크기를 조절해서 여러개의 파일로 분할해서 사용가능
# TFRecord를 만들려면 Protocol Buffer라고 불리는 Tensorflow에서 제공하는 직렬화 기법 이용
# 직렬화는 프로그램 객체를 저장가능한 형태의 데이터로 변환 시키는 작업을 지칭
# CSV, JSON, XML같은 것들이 바로 직렬화 방법

# 이미지 데이터 => Protocol Buffer => TFRecord
# Protocol Buffer를 이용해서 데이터를 어떻게 직렬화 할것인지 작성
# protoc라는 프로그램 필요. 이 protoc가 직렬화 해주는 프로그램.
# Google Colab은 이미 protoc가 이미 설치 되어있음
# 대신 PC나 AWS EC2같은 경우에는 이 protoc를 따로 설치 해야함

In [15]:
!ls object_detection/protos/

anchor_generator_pb2.py		       keypoint_box_coder_pb2.py
anchor_generator.proto		       keypoint_box_coder.proto
argmax_matcher_pb2.py		       losses_pb2.py
argmax_matcher.proto		       losses.proto
bipartite_matcher_pb2.py	       matcher_pb2.py
bipartite_matcher.proto		       matcher.proto
box_coder_pb2.py		       mean_stddev_box_coder_pb2.py
box_coder.proto			       mean_stddev_box_coder.proto
box_predictor_pb2.py		       model_pb2.py
box_predictor.proto		       model.proto
calibration_pb2.py		       multiscale_anchor_generator_pb2.py
calibration.proto		       multiscale_anchor_generator.proto
center_net_pb2.py		       optimizer_pb2.py
center_net.proto		       optimizer.proto
eval_pb2.py			       pipeline_pb2.py
eval.proto			       pipeline.proto
faster_rcnn_box_coder_pb2.py	       post_processing_pb2.py
faster_rcnn_box_coder.proto	       post_processing.proto
faster_rcnn_pb2.py		       preprocessor_pb2.py
faster_rcnn.proto		       preprocessor.proto
flexible_grid_anchor_generator_p

In [16]:
# protoc를 이용해서 컴파일 처리
# *.proto => 모든 프로토 파일을 컴파일
!protoc object_detection/protos/*.proto --python_out=.

In [17]:
!ls object_detection/protos/

anchor_generator_pb2.py		       keypoint_box_coder_pb2.py
anchor_generator.proto		       keypoint_box_coder.proto
argmax_matcher_pb2.py		       losses_pb2.py
argmax_matcher.proto		       losses.proto
bipartite_matcher_pb2.py	       matcher_pb2.py
bipartite_matcher.proto		       matcher.proto
box_coder_pb2.py		       mean_stddev_box_coder_pb2.py
box_coder.proto			       mean_stddev_box_coder.proto
box_predictor_pb2.py		       model_pb2.py
box_predictor.proto		       model.proto
calibration_pb2.py		       multiscale_anchor_generator_pb2.py
calibration.proto		       multiscale_anchor_generator.proto
center_net_pb2.py		       optimizer_pb2.py
center_net.proto		       optimizer.proto
eval_pb2.py			       pipeline_pb2.py
eval.proto			       pipeline.proto
faster_rcnn_box_coder_pb2.py	       post_processing_pb2.py
faster_rcnn_box_coder.proto	       post_processing.proto
faster_rcnn_pb2.py		       preprocessor_pb2.py
faster_rcnn.proto		       preprocessor.proto
flexible_grid_anchor_generator_p

In [18]:
# pretrained_model이라는 폴더를 /content/TF2_Object_Detection_API 여기 밑에 생성
!mkdir /content/TF2_Object_Detection_API/pretrained_model

mkdir: cannot create directory ‘/content/TF2_Object_Detection_API/pretrained_model’: File exists


In [19]:
# 기본적인 준비가 됐으면
# 데이터셋을 준비
# 처음은 잘 알려진 데이터셋을 사용
# COCO dataset, PASCAL VOC, oxford PET dataset 등등
# PASCAL VOC dataset을 이용
# 각 데이터셋은 그 데이터를 표현하는 방법이 각각 다름

In [20]:
# 일단 PASCAL VOC dataset 다운로드
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

--2023-10-10 13:00:11--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2012.tar.3’

VOCtrainval_11-May- 100%[===================>]   1.86G   118MB/s    in 13s     

2023-10-10 13:00:23 (151 MB/s) - ‘VOCtrainval_11-May-2012.tar.3’ saved [1999639040/1999639040]



In [21]:
# 다운받은 파일 압출 풀기
!tar -xvf VOCtrainval_11-May-2012.tar

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
VOCdevkit/VOC2012/SegmentationClass/2008_001874.png
VOCdevkit/VOC2012/SegmentationClass/2008_001876.png
VOCdevkit/VOC2012/SegmentationClass/2008_001882.png
VOCdevkit/VOC2012/SegmentationClass/2008_001885.png
VOCdevkit/VOC2012/SegmentationClass/2008_001895.png
VOCdevkit/VOC2012/SegmentationClass/2008_001896.png
VOCdevkit/VOC2012/SegmentationClass/2008_001926.png
VOCdevkit/VOC2012/SegmentationClass/2008_001966.png
VOCdevkit/VOC2012/SegmentationClass/2008_001971.png
VOCdevkit/VOC2012/SegmentationClass/2008_001992.png
VOCdevkit/VOC2012/SegmentationClass/2008_001997.png
VOCdevkit/VOC2012/SegmentationClass/2008_002032.png
VOCdevkit/VOC2012/SegmentationClass/2008_002043.png
VOCdevkit/VOC2012/SegmentationClass/2008_002064.png
VOCdevkit/VOC2012/SegmentationClass/2008_002066.png
VOCdevkit/VOC2012/SegmentationClass/2008_002067.png
VOCdevkit/VOC2012/SegmentationClass/2008_002073.png
VOCdevkit/VOC2012/SegmentationClass/2008_002079.png
VOCdevkit/VOC2012/Segmentati

In [22]:
# TFRecord를 만들기 위해서 필요한 환경변수 하나 설정
import os
os.environ['PYTHONPATH'] = "/content/TF2_Object_Detection_API/models/research"

In [23]:
# TFRecord를 생성
# 2개 생성(train, validation)
!python object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path=object_detection/data/pascal_label_map.pbtxt \
    --data_dir=VOCdevkit --year=VOC2012 --set=train \
    --output_path=pascal_train.record

2023-10-10 13:00:52.660192: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 13:00:52.660235: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 13:00:52.663720: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 13:00:53.106525: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 13:00:55.205448: W tensorflow/c

In [24]:
!python object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path=object_detection/data/pascal_label_map.pbtxt \
    --data_dir=VOCdevkit --year=VOC2012 --set=val \
    --output_path=pascal_val.record

2023-10-10 13:01:08.330522: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 13:01:08.330580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 13:01:08.330629: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 13:01:08.342435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 13:01:10.206409: W tensorflow/c

In [25]:
# 데이터셋 준비 완료
# 학습을 진행하기 위해서 디렉토리 구조를 만들어야 함
# 디렉토르 구조는 PDF파일에 표현해놨음 1010파일

!mkdir /content/TF2_Object_Detection_API/my_od_model

mkdir: cannot create directory ‘/content/TF2_Object_Detection_API/my_od_model’: File exists


In [26]:
cd /content/TF2_Object_Detection_API/my_od_model

/content/TF2_Object_Detection_API/my_od_model


In [27]:
!mkdir /content/TF2_Object_Detection_API/my_od_model/data

mkdir: cannot create directory ‘/content/TF2_Object_Detection_API/my_od_model/data’: File exists


In [28]:
!mkdir /content/TF2_Object_Detection_API/my_od_model/models

mkdir: cannot create directory ‘/content/TF2_Object_Detection_API/my_od_model/models’: File exists


In [29]:
# 이제 /content/TF2_Object_Detection_API/my_od_model/data 폴더에 TFRecord파일 2개를 복사해서 넣기
# map파일도 같이 넣음
!cp /content/TF2_Object_Detection_API/models/research/pascal_train.record /content/TF2_Object_Detection_API/my_od_model/data

In [30]:
!cp /content/TF2_Object_Detection_API/models/research/pascal_val.record /content/TF2_Object_Detection_API/my_od_model/data

In [31]:
!cp /content/TF2_Object_Detection_API/models/research/object_detection/data/pascal_label_map.pbtxt /content/TF2_Object_Detection_API/my_od_model/data

In [32]:
# models directory안에 폴더 하나 생성
!mkdir /content/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model

mkdir: cannot create directory ‘/content/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model’: File exists


In [33]:
# 모델 학습을 위해 configuration파일 만들기
# pretrained model의 weight파일 필요

In [34]:
# weight 파일을 얻기 위해 아래 코드를 실행
# Faster R-CNN ResNet 101 V1 800x1333

import os
import six.moves.urllib as urllib
import tarfile

BASE_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
MODEL_NAME = 'faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8'
MODEL_FILE = MODEL_NAME + '.tar.gz'

DOWNLOAD_URL = BASE_URL + MODEL_FILE
DOWNLOAD_PATH = '/content/TF2_Object_Detection_API/pretrained_model'

if not os.path.isdir(DOWNLOAD_PATH):
    os.mkdir(DOWNLOAD_PATH)

opener = urllib.request.URLopener()
opener.addheader('User-Agent', 'ballentain')
opener.retrieve(DOWNLOAD_URL, filename = DOWNLOAD_PATH + '/' + MODEL_FILE)
print('모델 다운로드 완료...')

tar_file = tarfile.open(DOWNLOAD_PATH + '/' + MODEL_FILE)
tar_file.extractall(DOWNLOAD_PATH)
print('압축 해제 완료...')

모델 다운로드 완료...
압축 해제 완료...


In [35]:
!cp -r /content/TF2_Object_Detection_API/pretrained_model/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint /content/TF2_Object_Detection_API/my_od_model

In [36]:
# 학습에 필요한 설정파일 만들기
# my_model.config라는 이름으로 파일 만들기 마우스로 직접 파일 만들기해서 이름 바꾸기 해야함

# Faster R-CNN with Resnet-101 (v1),
# Initialized from Imagenet classification checkpoint
#
# Train on GPU-8
#
# Achieves 36.6 mAP on COCO17 val


# 설정 파일
'''
model {
  faster_rcnn {
    num_classes: 20
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 800
        max_dimension: 1333
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101_keras'
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  # AWS 성능상 batch_size를 4로 설정해도 OOM 오류가 발생.
  batch_size: 1
  # AWS 성능상 학습에 걸리는 시간이 1step당 0.6초가 걸림
  # 200,000 step이면 약 33.3 시간이 걸리게 되니 step의 수를
  # 조절할 필요가 있음.
  num_steps: 10000
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: 0.01
          total_steps: 10000
          warmup_learning_rate: 0.0
          warmup_steps: 5000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }

  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint: "/content/TF2_Object_Detection_API/my_od_model/checkpoint/ckpt-0"
  fine_tune_checkpoint_type: "detection"
  from_detection_checkpoint: true
  data_augmentation_options {
    random_horizontal_flip {
    }
  }

  data_augmentation_options {
    random_adjust_hue {
    }
  }

  data_augmentation_options {
    random_adjust_contrast {
    }
  }

  data_augmentation_options {
    random_adjust_saturation {
    }
  }

  data_augmentation_options {
     random_square_crop_by_scale {
      scale_min: 0.6
      scale_max: 1.3
    }
  }
}

train_input_reader: {
  label_map_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_label_map.pbtxt"
  tf_record_input_reader {
    input_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_train.record"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_val.record"
  }
}
'''

SyntaxError: ignored

In [37]:
cd /content/TF2_Object_Detection_API/models/research

/content/TF2_Object_Detection_API/models/research


In [38]:
# Tensorflow version 확인
import tensorflow as tf
print(tf.__version__)   # 2.14.0

2.14.0


In [39]:
# 학습을 위해 tensorflow버전을 2.13으로 대체
!pip install tensorflow==2.13.0

  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.14.0
    Uninstalling tensorflow-2.14.0:
      Successfully uninstalled tensorflow-2.14.0
ER

In [40]:
# kernel restart 후
# working directory를 /content/TF2_Object_Detection_API/models/research로 변경

In [41]:
cd /content/TF2_Object_Detection_API/models/research

/content/TF2_Object_Detection_API/models/research


In [42]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model/my_model.config \
    --model_dir=/content/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model \
    --alsologtostderr

2023-10-10 13:13:38.504209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-10 13:13:44.974704: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1010 13:13:44.978932 137963466838656 mirrored_strategy.py:419] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1010 13:13:45.004830 137963466838656 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1010 13:13:45.005035 137963466838656 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W1010 13:13:45.043920 137963466838656 dep